In [ ]:
from functools import wraps

import pyvista as pv

from ipyvtk_simple.viewer import ViewInteractiveWidget


class iPlotter(pv.Plotter):
    """Wrapping of PyVista's Plotter to be used interactively in Jupyter."""

    def __init__(self, *args, **kwargs):
        transparent_background = kwargs.pop('transparent_background', pv.rcParams['transparent_background'])
        kwargs["notebook"] = False
        kwargs["off_screen"] = False
        pv.Plotter.__init__(self, *args, **kwargs)
        self.ren_win.SetOffScreenRendering(1)
        self.off_screen = True
        self._widget = ViewInteractiveWidget(self.ren_win, transparent_background=transparent_background)


    @wraps(pv.Plotter.show)
    def show(self, *args, **kwargs):
        kwargs["auto_close"] = False
        _ = pv.Plotter.show(self, *args, **kwargs) # Incase the user sets the cpos or something
        return self.widget

    @property
    def widget(self):
        self._widget.full_render()
        return self._widget

In [ ]:
mesh = pv.Wavelet()

p = iPlotter()
p.add_mesh_clip_box(mesh)
p.show()

In [ ]:
from pyvista import examples

volume = examples.download_knee_full()
opacity = [0, 0, 0, 0.1, 0.3, 0.6, 1]

plotter = iPlotter()
plotter.add_volume(volume, opacity=opacity)
plotter.show()

In [ ]:
topo = examples.load_random_hills()

# create a subset of arrows using the glyph filter
arrows = topo.glyph(scale="Normals", orient="Normals", tolerance=0.05)

p = iPlotter()
p.add_mesh(arrows, color="black")
p.add_mesh(topo, scalars="Elevation", cmap="terrain")
p.show()